In [35]:
import random
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab

/Users/konglingkun/anaconda/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [37]:
# here, coin number is saved as float (complying with code from github, while contradicts white paper)
class Smartcoin(object):
	def __init__(self, name='kennycoin', reservetokenName='aCoin', initCRR=0.5, initPrice=1, initIssueNum=100):
		self._Name = name
		self._ReservetokenName = reservetokenName
		self._CRR = float(initCRR)
		self._Price = float(initPrice)
		self._Supply = float(initIssueNum)
		self._ReserveBalance = float(initCRR * initIssueNum)
		self._budget = float(0)

	def printInfo(self):
		print '---------'
		print 'NAME:', self._Name, '| RESERVE NAME:', self._ReservetokenName, 'CRR:', self._CRR
		print 'PRICE:',self._Price
		print 'SUPPLY:', self._Supply, '| RESERVE BALANCE:', self._ReserveBalance
		print 'BUDGET:', self._budget
        
	def updatePrice(self, reserveBalance, supply, CRR):
		newPrice = reserveBalance/(supply * CRR)
		return newPrice
    
	def setCRR(self, newCRR = 0.5):
		oldCRR = self._CRR
		self._CRR = newCRR
		print 'CRR', oldCRR, '->', newCRR

	def getPrice(self):
		return self._Price

	def purchasing(self, convertIntoNum=0):
		issuedtokenNum = self._Supply * (((self._ReserveBalance + convertIntoNum)/self._ReserveBalance)**(self._CRR) - 1)
		self._Supply = self._Supply + issuedtokenNum
		self._ReserveBalance = self._ReserveBalance + convertIntoNum
		oldPrice = self._Price
		self._Price = self.updatePrice(self._ReserveBalance, self._Supply, self._CRR)
		increaseRatio = (self._Price - oldPrice)/oldPrice
		print '*********'
		print convertIntoNum, self._ReservetokenName+" convert into "+self._Name
		print "Current prize of "+self._Name+" is", self._Price, "with", increaseRatio, "increasing."
		self._budget = self._budget + self._Price*self._Supply - oldPrice*(self._Supply - issuedtokenNum)

	def destroying(self, convertOutNum=0):
		destroyedtokenNum = self._Supply * (1 - ((self._ReserveBalance - convertOutNum)/self._ReserveBalance)**(self._CRR))
		self._Supply = self._Supply - destroyedtokenNum
		self._ReserveBalance = self._ReserveBalance - convertOutNum
		oldPrice = self._Price
		self._Price = self.updatePrice(self._ReserveBalance, self._Supply, self._CRR)
		decreaseRatio = (oldPrice-self._Price)/oldPrice
		print '*********'
		print convertOutNum, self._ReservetokenName+" convert out from "+self._Name
		print "Current prize of "+self._Name+" is", self._Price, "with", decreaseRatio, "decreasing."
		self._budget = self._budget + self._Price*self._Supply - oldPrice*(self._Supply + destroyedtokenNum)

SyntaxError: invalid syntax (<ipython-input-37-4cb82e94e91f>, line 2)

In [30]:
# example of using smartcoin class
BNTCoin = Smartcoin(name='BNT',reservetokenName='ETH',initCRR=0.2, initPrice=1,initIssueNum=300000)
BNTCoin.printInfo()
BNTCoin.purchasing(300)
BNTCoin.printInfo()
BNTCoin.purchasing(700)
BNTCoin.printInfo()
BNTCoin.destroying(1308)
BNTCoin.printInfo()
BNTCoin.purchasing(100)
BNTCoin.printInfo()

---------
NAME: BNT | RESERVE NAME: ETH CRR: 0.2
PRICE: 1.0
SUPPLY: 300000.0 | RESERVE BALANCE: 60000.0
BUDGET: 0.0
*********
300 ETH convert into BNT
Current prize of BNT is 1.00399800399 with 0.00399800398904 increasing.
---------
NAME: BNT | RESERVE NAME: ETH CRR: 0.2
PRICE: 1.00399800399
SUPPLY: 300299.401794 | RESERVE BALANCE: 60300.0
BUDGET: 1500.0
*********
700 ETH convert into BNT
Current prize of BNT is 1.01331125792 with 0.00927616777077 increasing.
---------
NAME: BNT | RESERVE NAME: ETH CRR: 0.2
PRICE: 1.01331125792
SUPPLY: 300993.399232 | RESERVE BALANCE: 61000.0
BUDGET: 5000.0
*********
1308 ETH convert out from BNT
Current prize of BNT is 0.995891220904 with 0.0171912005081 decreasing.
---------
NAME: BNT | RESERVE NAME: ETH CRR: 0.2
PRICE: 0.995891220904
SUPPLY: 299691.365618 | RESERVE BALANCE: 59692.0
BUDGET: -1540.0
*********
100 ETH convert into BNT
Current prize of BNT is 0.997225703909 with 0.0013399887228 increasing.
---------
NAME: BNT | RESERVE NAME: ETH CRR: 0.

In [31]:
class Customers(object):
    def __init__(self, smartToken, smartValue = 0, reserveValue = 500):
        # _smartToken is the token customers want to buy -- SmartToken()
        self._smartToken = smartToken
        # _ownedSmartToken refers to smart token's number customers have
        self._ownedSmartToken = float(0)
        # _smartValue is the value of ownedSmartTokens -- currentPrize * owned#
        self._smartValue = float(0)
        self._reserveValue = float(reserveValue)
        # _ownedvalue = _reserveValue + _smartValue
        self._ownedvalue = float(reserveValue)
        # gain or lose money comparing to the original state
        self._budget = float(0)
        
    def printinfo(self):
        oldownedValue = self._ownedvalue
        self._ownedvalue = self._smartToken._Price * self._ownedSmartToken + self._reserveValue
        self._budget = self._budget + self._ownedvalue - oldownedValue
        print '------'
        print 'smartToken Name:', self._smartToken._Name
        print 'ownedvalue:', self._ownedvalue, 'budget:', self._budget
        
    def purchase(self, TokenNumber):
        oldPrice = self._smartToken._Price
        if oldPrice * TokenNumber > self._reserveValue:
            print 'Invalid Operation in Purchase'
            return
        self._ownedSmartToken = self._ownedSmartToken + TokenNumber
        self._reserveValue = self._reserveValue - oldPrice * TokenNumber
        # smartToken.purchasing means being purchased
        self._smartToken.purchasing(TokenNumber)
        newPrice = self._smartToken._Price
        self._smartValue = self._ownedSmartToken * newPrice
        oldownedValue = self._ownedvalue
        self._ownedvalue = self._reserveValue + self._smartValue
        self._budget = self._budget + self._ownedvalue - oldownedValue
        
        
    def destroy(self, TokenNumber):
        oldPrice = self._smartToken._Price
        if self._ownedSmartToken < TokenNumber:
            print 'Invalid Operation in Destroy'
            return
        self._ownedSmartToken = self._ownedSmartToken - TokenNumber
        self._reserveValue = self._reserveValue + oldPrice*TokenNumber
        # smartToken.purchasing means being purchased
        self._smartToken.destroying(TokenNumber)
        newPrice = self._smartToken._Price
        self._smartValue = self._ownedSmartToken*newPrice
        oldownedValue = self._ownedvalue
        self._ownedvalue = self._reserveValue + self._smartValue
        self._budget = self._budget + self._ownedvalue - oldownedValue
        

In [32]:
KennyCoin = Smartcoin(name='Kenny',reservetokenName='ETH',initCRR=0.2, initPrice=1,initIssueNum=300000)

In [33]:
Alice = Customers(smartToken=KennyCoin)
Alice.purchase(100)
Alice.printinfo()
KennyCoin.printInfo()
Bob = Customers(smartToken=KennyCoin)
Bob.purchase(100)
Bob.printinfo()
Alice.printinfo()
# KennyCoin.printInfo()

*********
100 ETH convert into Kenny
Current prize of Kenny is 1.00133311126 with 0.00133311125912 increasing.
------
smartToken Name: Kenny
ownedvalue: 500.133311126 budget: 0.133311125912
---------
NAME: Kenny | RESERVE NAME: ETH CRR: 0.2
PRICE: 1.00133311126
SUPPLY: 300099.9334 | RESERVE BALANCE: 60100.0
BUDGET: 500.0
*********
100 ETH convert into Kenny
Current prize of Kenny is 1.00266577896 with 0.0013308934726 increasing.
------
smartToken Name: Kenny
ownedvalue: 500.13326677 budget: 0.133266770167
------
smartToken Name: Kenny
ownedvalue: 500.266577896 budget: 0.266577896079


In [36]:
for x in xrange(0,100):
    seed = random.randint(1, 10)
    if seed <= 5:
        # people purchasing
        pass
    else:
        # people destroying coins
        pass